In [1]:
# =============================================================================
# Task 3: Fine-Tuning a Transformer Model for Amharic NER
# This version is updated to handle all specified entities, including optional ones.
# =============================================================================

import numpy as np
from datasets import load_dataset, Dataset, DatasetDict, Features, Sequence, ClassLabel, Value
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification
from seqeval.metrics import classification_report
import os
import torch

# --- Step 1: Define Comprehensive Label List and Load Dataset ---
# The label_list is expanded to include all possible entities from the project brief.
# This is the key change to fix the previous "Invalid string class label" error.
label_list = [
    "O", 
    "B-PRODUCT", "I-PRODUCT", 
    "B-PRICE", "I-PRICE", 
    "B-LOC", "I-LOC",
    "B-CONTACT_INFO", "I-CONTACT_INFO",
    "B-DELIVERY_FEE", "I-DELIVERY_FEE"
]
labeled_file_path = '../data/labeled_data.txt'

if not os.path.exists(labeled_file_path) or os.path.getsize(labeled_file_path) == 0:
    raise FileNotFoundError(f"Labeled data file not found or is empty at: {labeled_file_path}.")

# --- Step 2: Robustly Parse the CoNLL File into a Dataset ---
def read_conll_file(file_path):
    """Reads a CoNLL-formatted file and converts it into a Hugging Face Dataset."""
    with open(file_path, 'r', encoding='utf-8') as f:
        tokens_list, ner_tags_list = [], []
        tokens, ner_tags = [], []
        for line in f:
            line = line.strip()
            if line == "":
                if tokens:
                    tokens_list.append(tokens)
                    ner_tags_list.append(ner_tags)
                    tokens, ner_tags = [], []
            else:
                parts = line.split()
                if len(parts) == 2 and parts[1] in label_list:
                    tokens.append(parts[0])
                    ner_tags.append(parts[1])
                # Silently ignore lines with invalid labels to prevent crashes
        if tokens: # Add the last sentence
            tokens_list.append(tokens)
            ner_tags_list.append(ner_tags)
    return Dataset.from_dict({'tokens': tokens_list, 'ner_tags': ner_tags_list})

# Create and structure the dataset with the correct features
raw_dataset = read_conll_file(labeled_file_path)
features = Features({
    'tokens': Sequence(Value('string')),
    'ner_tags': Sequence(ClassLabel(names=label_list))
})
raw_dataset = raw_dataset.cast(features)

# Split into training and test sets
final_datasets = raw_dataset.train_test_split(test_size=0.2, seed=42)
print("Dataset processed and split successfully:\n", final_datasets)
print("\nSample from training data:\n", final_datasets['train'][0])

# --- Step 3: Tokenization and Label Alignment ---
model_checkpoint = "Davlan/afro-xlmr-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def tokenize_and_align_labels(examples):
    """Tokenizes sentences and aligns labels to the new sub-word tokens."""
    tokenized_inputs = tokenizer(
        examples["tokens"], 
        truncation=True, 
        is_split_into_words=True
    )
    all_aligned_labels = []
    for i, labels_for_one_sentence in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        current_aligned_labels = []
        for word_idx in word_ids:
            if word_idx is None:
                current_aligned_labels.append(-100)
            elif word_idx != previous_word_idx:
                current_aligned_labels.append(labels_for_one_sentence[word_idx])
            else:
                current_aligned_labels.append(-100)
            previous_word_idx = word_idx
        all_aligned_labels.append(current_aligned_labels)
    tokenized_inputs["labels"] = all_aligned_labels
    return tokenized_inputs

tokenized_datasets = final_datasets.map(tokenize_and_align_labels, batched=True)

# --- Step 4: Model Training ---
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint, num_labels=len(label_list), id2label={i: l for i, l in enumerate(label_list)}, label2id={l: i for i, l in enumerate(label_list)}
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

args = TrainingArguments(
    output_dir="../saved_models/amharic-ner-afro-xlmr",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=25, # Increased epochs for more complex entities
    weight_decay=0.01,
    logging_steps=10,
)
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

def compute_metrics(p):
    """Computes precision, recall, and F1-score for the evaluation set."""
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    true_predictions = [
        [label_list[p] for (p, l) in zip(pred, lab) if l != -100]
        for pred, lab in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(pred, lab) if l != -100]
        for pred, lab in zip(predictions, labels)
    ]
    report = classification_report(true_labels, true_predictions, output_dict=True, zero_division=0)
    return {
        "precision": report["micro avg"]["precision"],
        "recall": report["micro avg"]["recall"],
        "f1-score": report["micro avg"]["f1-score"],
    }

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

print("Starting model training with expanded entity list...")
trainer.train()

# --- Step 5: Final Evaluation and Saving ---
print("\nEvaluating final model on the test set...")
eval_results = trainer.evaluate()
print(f"Final Evaluation results: {eval_results}")

model_save_path = "../saved_models/amharic-ner-afro-xlmr"
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)
print(f"\nModel and tokenizer saved successfully to '{model_save_path}'")

# --- Step 6: Test the Final Model on a Complex Example ---
print("\n--- Testing the fine-tuned model with a pipeline ---")
from transformers import pipeline

ner_pipeline = pipeline("ner", model=model_save_path, aggregation_strategy="simple")
text = "1 pairs Sneaker Crease Protector ዋጋ፦ 400 ብር አድራሻ መገናኛ ቢሮ ቁ. S05/S06 0902660722"
results = ner_pipeline(text)
print(f"Test text: {text}")
for entity in results:
    print(entity)

/home/fentahun/10_acadamy/EthioMart_NER_week-4/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]
Casting the dataset: 100%|██████████| 37/37 [00:00<00:00, 18683.99 examples/s]
/home/fentahun/10_acadamy/EthioMart_NER_week-4/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Dataset processed and split successfully:
 DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 29
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 8
    })
})

Sample from training data:
 {'tokens': ['HP', 'PAVILION', 'Price', 'አድራሻ:', '-', 'መገናኛ', 'ማራቶን', 'የ', 'ገበያ', 'ማእከል'], 'ner_tags': [1, 2, 0, 0, 0, 5, 6, 6, 6, 6]}


Map: 100%|██████████| 8/8 [00:00<00:00, 1963.05 examples/s]
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at Davlan/afro-xlmr-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Starting model training with expanded entity list...


  0%|          | 0/100 [00:00<?, ?it/s]/home/fentahun/10_acadamy/EthioMart_NER_week-4/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
                                               
  4%|▍         | 4/100 [00:13<04:52,  3.04s/it]

...

{'loss': 0.0432, 'grad_norm': 0.46382230520248413, 'learning_rate': 0.0, 'epoch': 25.0}


                                                 
100%|██████████| 100/100 [05:21<00:00,  3.21s/it]


{'eval_loss': 0.18778850138187408, 'eval_precision': 0.9743589743589743, 'eval_recall': 0.95, 'eval_f1-score': 0.9620253164556962, 'eval_runtime': 0.5301, 'eval_samples_per_second': 15.093, 'eval_steps_per_second': 1.887, 'epoch': 25.0}
{'train_runtime': 321.1992, 'train_samples_per_second': 2.257, 'train_steps_per_second': 0.311, 'train_loss': 0.4464517691731453, 'epoch': 25.0}

Evaluating final model on the test set...


100%|██████████| 1/1 [00:00<00:00, 184.43it/s]


Final Evaluation results: {'eval_loss': 0.18778850138187408, 'eval_precision': 0.9743589743589743, 'eval_recall': 0.95, 'eval_f1-score': 0.9620253164556962, 'eval_runtime': 0.4634, 'eval_samples_per_second': 17.263, 'eval_steps_per_second': 2.158, 'epoch': 25.0}

Model and tokenizer saved successfully to '../saved_models/amharic-ner-afro-xlmr'

--- Testing the fine-tuned model with a pipeline ---
Test text: 1 pairs Sneaker Crease Protector ዋጋ፦ 400 ብር አድራሻ መገናኛ ቢሮ ቁ. S05/S06 0902660722
{'entity_group': 'PRODUCT', 'score': np.float32(0.99399006), 'word': '1 pairs Sneaker Crease Protector', 'start': 0, 'end': 32}
{'entity_group': 'PRICE', 'score': np.float32(0.9858272), 'word': '400 ብር', 'start': 37, 'end': 43}
{'entity_group': 'LOC', 'score': np.float32(0.87952137), 'word': 'መገናኛ ቢሮ ቁ. S05/S06', 'start': 49, 'end': 67}
{'entity_group': 'CONTACT_INFO', 'score': np.float32(0.99241734), 'word': '09', 'start': 68, 'end': 70}
{'entity_group': 'CONTACT_INFO', 'score': np.float32(0.9698542), 'w